# PRODUCTION-GRADE Speech Analytics Platform with 2025 SOTA Techniques
## Incorporating latest research: emotion2vec+, Vision Transformers, Graph Neural Networks
### Author: Peter Chika Ozo-ogueji (Data Scientist)

In [1]:
#!/usr/bin/env python3
"""
CLEAN AUDIO-ONLY SOTA Speech Analytics - NO SYNTHETIC FEATURES
Real audio signal processing features only for reliable deployment
Author: Peter Chika Ozo-ogueji (Data Scientist)

FIXED: Removes Vision Transformer, Graph, and Quantum features
Target: 80%+ accuracy with deployable reliability
"""

print("🚀 CLEAN AUDIO-ONLY SOTA SPEECH ANALYTICS")
print("🎯 NO SYNTHETIC FEATURES - RELIABLE DEPLOYMENT")
print("👨‍💻 Author: Peter Chika Ozo-ogueji (Data Scientist)")
print("🔧 FIXED: Removes problematic synthetic features")
print("=" * 80)

# Install packages
import subprocess
import sys

def install_packages():
    packages = [
        'kaggle', 'librosa', 'soundfile', 'scikit-learn', 'plotly',
        'matplotlib', 'seaborn', 'pandas', 'numpy', 'tqdm',
        'xgboost', 'lightgbm', 'imbalanced-learn',
        'scipy', 'audiomentations'
    ]

    for package in packages:
        try:
            subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", package])
        except:
            print(f"Warning: Could not install {package}")

install_packages()

# Import libraries
import os
import json
import numpy as np
import pandas as pd
import librosa
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.svm import SVC
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, f1_score
from sklearn.preprocessing import StandardScaler, LabelEncoder, RobustScaler
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.neural_network import MLPClassifier
import xgboost as xgb
import lightgbm as lgb
from imblearn.over_sampling import SMOTE, BorderlineSMOTE
import plotly.express as px
from tqdm import tqdm
import warnings
import re
import glob
from scipy import stats
import time
import gc
from typing import Dict, List, Tuple, Optional
from dataclasses import dataclass
from pathlib import Path

warnings.filterwarnings('ignore')

print("✅ All packages imported successfully!")

# =============================================================================
# 1. KAGGLE SETUP (Same as before)
# =============================================================================

def setup_kaggle():
    """Setup Kaggle API"""
    print("🔑 Setting up Kaggle API...")

    kaggle_dirs = ['/root/.kaggle', '/root/.config/kaggle', os.path.expanduser('~/.kaggle')]
    for directory in kaggle_dirs:
        os.makedirs(directory, exist_ok=True)

    kaggle_credentials = {
        "username": "peterchikaozoogueji",
        "key": "f2b5ae97165cf3eef611db7624db7192"
    }

    for directory in kaggle_dirs:
        if os.path.exists(directory):
            kaggle_file = os.path.join(directory, 'kaggle.json')
            with open(kaggle_file, 'w') as f:
                json.dump(kaggle_credentials, f)
            os.chmod(kaggle_file, 0o600)

    print("✅ Kaggle API configured!")
    return True

# =============================================================================
# 2. DATASET DOWNLOADING (Same as before)
# =============================================================================

def download_sota_datasets():
    """Download proven audio emotion datasets"""
    print("\n📥 DOWNLOADING PROVEN AUDIO EMOTION DATASETS")
    print("🎯 Focus: Real audio signal processing only")
    print("=" * 60)

    base_path = '/content/clean_audio_datasets' if '/content' in os.getcwd() else './clean_audio_datasets'
    os.makedirs(base_path, exist_ok=True)
    os.makedirs(f'{base_path}/emotion', exist_ok=True)

    # Proven audio emotion datasets
    datasets = [
        {
            'name': 'RAVDESS Audio-Only',
            'kaggle_id': 'uwrfkaggler/ravdess-emotional-speech-audio',
            'path': f'{base_path}/emotion/ravdess/',
            'priority': 1,
            'accuracy': '85%+ with clean features'
        },
        {
            'name': 'CREMA-D Audio',
            'kaggle_id': 'dmitrybabko/speech-emotion-recognition-en',
            'path': f'{base_path}/emotion/crema_d/',
            'priority': 1,
            'accuracy': '80%+ validated'
        },
        {
            'name': 'TESS Clean Audio',
            'kaggle_id': 'ejlok1/toronto-emotional-speech-set-tess',
            'path': f'{base_path}/emotion/tess/',
            'priority': 1,
            'accuracy': 'High performance'
        },
        {
            'name': 'EMO-DB German',
            'kaggle_id': 'piyushagni5/berlin-database-of-emotional-speech-emodb',
            'path': f'{base_path}/emotion/emodb/',
            'priority': 2,
            'accuracy': '90%+ on clean features'
        },
        {
            'name': 'SAVEE British',
            'kaggle_id': 'barelydedicated/savee-database',
            'path': f'{base_path}/emotion/savee/',
            'priority': 2,
            'accuracy': 'Proven benchmark'
        }
    ]

    successful_downloads = []

    for dataset in datasets:
        try:
            print(f"\n📊 Downloading {dataset['name']}...")
            print(f"   🎯 Expected Performance: {dataset['accuracy']}")
            os.makedirs(dataset['path'], exist_ok=True)

            result = os.system(f"kaggle datasets download -d {dataset['kaggle_id']} -p {dataset['path']} --unzip")

            if result == 0 or os.listdir(dataset['path']):
                audio_files = []
                for root, dirs, files in os.walk(dataset['path']):
                    for file in files:
                        if file.lower().endswith(('.wav', '.mp3', '.flac', '.m4a', '.aac')):
                            audio_files.append(os.path.join(root, file))

                print(f"   ✅ Total audio files: {len(audio_files)}")

                successful_downloads.append({
                    'name': dataset['name'],
                    'path': dataset['path'],
                    'file_count': len(audio_files),
                    'files': audio_files,
                    'priority': dataset['priority'],
                    'accuracy': dataset['accuracy']
                })

        except Exception as e:
            print(f"   ❌ Error downloading {dataset['name']}: {e}")

    total_files = sum(d['file_count'] for d in successful_downloads)
    print(f"\n✅ Downloaded {len(successful_downloads)} proven datasets with {total_files:,} files!")

    return successful_downloads

# =============================================================================
# 3. CLEAN AUDIO-ONLY FEATURE EXTRACTION
# =============================================================================

class CleanAudioFeatureExtractor:
    """Clean audio feature extraction - NO SYNTHETIC FEATURES"""

    def __init__(self, sample_rate=22050):
        self.sample_rate = sample_rate
        self.feature_names = None

    def extract_clean_audio_features(self, audio_file_path):
        """Extract ONLY real audio signal processing features"""
        try:
            # Load audio
            audio, sr = librosa.load(audio_file_path, sr=self.sample_rate, duration=3.0)
            if audio is None or len(audio) == 0:
                return {}

            # Clean audio
            if not np.isfinite(audio).all():
                audio = np.nan_to_num(audio, nan=0.0, posinf=0.0, neginf=0.0)

            if np.max(np.abs(audio)) > 0:
                audio = librosa.util.normalize(audio)

            features = {}

            # 1. COMPREHENSIVE MFCC FEATURES (104 features)
            features.update(self._extract_mfcc_features(audio, sr))

            # 2. SPECTRAL FEATURES (16 features)
            features.update(self._extract_spectral_features(audio, sr))

            # 3. CHROMA FEATURES (24 features)
            features.update(self._extract_chroma_features(audio, sr))

            # 4. PROSODIC FEATURES (11 features)
            features.update(self._extract_prosodic_features(audio, sr))

            # 5. ADVANCED SPECTRAL FEATURES (16 features)
            features.update(self._extract_advanced_spectral_features(audio, sr))

            # 6. HARMONIC FEATURES (15 features)
            features.update(self._extract_harmonic_features(audio, sr))

            # 7. TEMPORAL FEATURES (5 features)
            features.update(self._extract_temporal_features(audio, sr))

            # Clean all features
            for key, value in features.items():
                if np.isnan(value) or np.isinf(value):
                    features[key] = 0.0

            if self.feature_names is None:
                self.feature_names = list(features.keys())
                print(f"✅ Extracting {len(self.feature_names)} CLEAN audio features per file")

            return features

        except Exception as e:
            if self.feature_names is not None:
                return {name: 0.0 for name in self.feature_names}
            return {}

    def _extract_mfcc_features(self, audio, sr):
        """Comprehensive MFCC features - most important for emotion recognition"""
        features = {}

        try:
            # Enhanced MFCC extraction
            mfccs = librosa.feature.mfcc(y=audio, sr=sr, n_mfcc=13, n_fft=2048, hop_length=512)
            mfcc_delta = librosa.feature.delta(mfccs)
            mfcc_delta2 = librosa.feature.delta(mfccs, order=2)

            for i in range(13):
                # Comprehensive statistics for each MFCC coefficient
                features[f'mfcc_{i}_mean'] = float(np.mean(mfccs[i]))
                features[f'mfcc_{i}_std'] = float(np.std(mfccs[i]))
                features[f'mfcc_{i}_max'] = float(np.max(mfccs[i]))
                features[f'mfcc_{i}_min'] = float(np.min(mfccs[i]))
                features[f'mfcc_{i}_skew'] = float(stats.skew(mfccs[i]))
                features[f'mfcc_{i}_kurtosis'] = float(stats.kurtosis(mfccs[i]))
                features[f'mfcc_delta_{i}_mean'] = float(np.mean(mfcc_delta[i]))
                features[f'mfcc_delta2_{i}_mean'] = float(np.mean(mfcc_delta2[i]))

        except Exception as e:
            # Fallback MFCC features
            for i in range(13):
                for stat in ['mean', 'std', 'max', 'min', 'skew', 'kurtosis']:
                    features[f'mfcc_{i}_{stat}'] = 0.0
                features[f'mfcc_delta_{i}_mean'] = 0.0
                features[f'mfcc_delta2_{i}_mean'] = 0.0

        return features

    def _extract_spectral_features(self, audio, sr):
        """Basic spectral features"""
        features = {}

        try:
            spectral_centroids = librosa.feature.spectral_centroid(y=audio, sr=sr)[0]
            spectral_rolloff = librosa.feature.spectral_rolloff(y=audio, sr=sr)[0]
            spectral_bandwidth = librosa.feature.spectral_bandwidth(y=audio, sr=sr)[0]
            zero_crossing_rate = librosa.feature.zero_crossing_rate(audio)[0]

            for name, feature_array in [
                ('spectral_centroid', spectral_centroids),
                ('spectral_rolloff', spectral_rolloff),
                ('spectral_bandwidth', spectral_bandwidth),
                ('zero_crossing_rate', zero_crossing_rate)
            ]:
                features[f'{name}_mean'] = float(np.mean(feature_array))
                features[f'{name}_std'] = float(np.std(feature_array))
                features[f'{name}_max'] = float(np.max(feature_array))
                features[f'{name}_skew'] = float(stats.skew(feature_array))

        except Exception as e:
            for name in ['spectral_centroid', 'spectral_rolloff', 'spectral_bandwidth', 'zero_crossing_rate']:
                for stat in ['mean', 'std', 'max', 'skew']:
                    features[f'{name}_{stat}'] = 0.0

        return features

    def _extract_chroma_features(self, audio, sr):
        """Chroma features for harmonic content"""
        features = {}

        try:
            chroma = librosa.feature.chroma_stft(y=audio, sr=sr, n_chroma=12)
            for i in range(12):
                features[f'chroma_{i}_mean'] = float(np.mean(chroma[i]))
                features[f'chroma_{i}_std'] = float(np.std(chroma[i]))

        except Exception as e:
            for i in range(12):
                features[f'chroma_{i}_mean'] = 0.0
                features[f'chroma_{i}_std'] = 0.0

        return features

    def _extract_prosodic_features(self, audio, sr):
        """Prosodic features (F0, energy, etc.)"""
        features = {}

        try:
            # F0 extraction (fixed - no threshold parameter)
            f0 = librosa.yin(audio, fmin=50, fmax=400)
            f0_clean = f0[f0 > 0]

            if len(f0_clean) > 0:
                features['f0_mean'] = float(np.mean(f0_clean))
                features['f0_std'] = float(np.std(f0_clean))
                features['f0_range'] = float(np.max(f0_clean) - np.min(f0_clean))
                features['f0_jitter'] = float(np.mean(np.abs(np.diff(f0_clean))) / np.mean(f0_clean)) if len(f0_clean) > 1 else 0.0
                features['f0_shimmer'] = float(np.std(f0_clean) / np.mean(f0_clean)) if np.mean(f0_clean) > 0 else 0.0
                features['f0_slope'] = float(np.polyfit(range(len(f0_clean)), f0_clean, 1)[0]) if len(f0_clean) > 1 else 0.0
                features['f0_curvature'] = float(np.polyfit(range(len(f0_clean)), f0_clean, 2)[0]) if len(f0_clean) > 2 else 0.0
            else:
                for feat in ['f0_mean', 'f0_std', 'f0_range', 'f0_jitter', 'f0_shimmer', 'f0_slope', 'f0_curvature']:
                    features[feat] = 0.0

            # Energy features
            rms = librosa.feature.rms(y=audio)[0]
            features['energy_mean'] = float(np.mean(rms))
            features['energy_std'] = float(np.std(rms))
            features['energy_skew'] = float(stats.skew(rms))
            features['energy_kurtosis'] = float(stats.kurtosis(rms))

        except Exception as e:
            for feat in ['f0_mean', 'f0_std', 'f0_range', 'f0_jitter', 'f0_shimmer',
                        'f0_slope', 'f0_curvature', 'energy_mean', 'energy_std', 'energy_skew', 'energy_kurtosis']:
                features[feat] = 0.0

        return features

    def _extract_advanced_spectral_features(self, audio, sr):
        """Advanced spectral features"""
        features = {}

        try:
            # Spectral contrast (7 bands × 2 stats = 14 features)
            spectral_contrast = librosa.feature.spectral_contrast(y=audio, sr=sr)
            for i in range(min(7, spectral_contrast.shape[0])):
                features[f'spectral_contrast_{i}_mean'] = float(np.mean(spectral_contrast[i]))
                features[f'spectral_contrast_{i}_std'] = float(np.std(spectral_contrast[i]))

            # Spectral flatness (2 features)
            spectral_flatness = librosa.feature.spectral_flatness(y=audio)[0]
            features['spectral_flatness_mean'] = float(np.mean(spectral_flatness))
            features['spectral_flatness_std'] = float(np.std(spectral_flatness))

        except Exception as e:
            for i in range(7):
                features[f'spectral_contrast_{i}_mean'] = 0.0
                features[f'spectral_contrast_{i}_std'] = 0.0
            features['spectral_flatness_mean'] = 0.0
            features['spectral_flatness_std'] = 0.0

        return features

    def _extract_harmonic_features(self, audio, sr):
        """Harmonic and tonal features"""
        features = {}

        try:
            # Tonnetz (6 × 2 = 12 features)
            tonnetz = librosa.feature.tonnetz(y=audio, sr=sr)
            for i in range(min(6, tonnetz.shape[0])):
                features[f'tonnetz_{i}_mean'] = float(np.mean(tonnetz[i]))
                features[f'tonnetz_{i}_std'] = float(np.std(tonnetz[i]))

            # Harmonic-percussive separation (3 features)
            y_harmonic, y_percussive = librosa.effects.hpss(audio)
            features['harmonic_energy'] = float(np.mean(y_harmonic**2))
            features['percussive_energy'] = float(np.mean(y_percussive**2))
            features['harmonic_percussive_ratio'] = float(features['harmonic_energy'] / (features['percussive_energy'] + 1e-8))

        except Exception as e:
            for i in range(6):
                features[f'tonnetz_{i}_mean'] = 0.0
                features[f'tonnetz_{i}_std'] = 0.0
            features['harmonic_energy'] = 0.0
            features['percussive_energy'] = 0.0
            features['harmonic_percussive_ratio'] = 0.0

        return features

    def _extract_temporal_features(self, audio, sr):
        """Temporal features (rhythm, beats, etc.)"""
        features = {}

        try:
            # Tempo and beat features
            tempo, beats = librosa.beat.beat_track(y=audio, sr=sr)
            features['tempo'] = float(tempo)
            features['beat_count'] = float(len(beats))
            features['beat_variance'] = float(np.var(np.diff(beats))) if len(beats) > 1 else 0.0

            # Onset detection
            onset_frames = librosa.onset.onset_detect(y=audio, sr=sr)
            features['onset_count'] = float(len(onset_frames))
            features['onset_rate'] = float(len(onset_frames) / (len(audio) / sr))

        except Exception as e:
            for feat in ['tempo', 'beat_count', 'beat_variance', 'onset_count', 'onset_rate']:
                features[feat] = 0.0

        return features

# =============================================================================
# 4. EMOTION EXTRACTION (Same as before)
# =============================================================================

class CleanEmotionExtractor:
    """Clean emotion extraction - same as before"""

    def __init__(self):
        self.debug_extractions = []
        self.extraction_stats = {'total': 0, 'success': 0, 'failed': 0}

        self.emotion_patterns = {
            'angry': ['angry', 'anger', 'ang', 'mad', 'furious', 'rage'],
            'calm': ['calm', 'neutral-calm', 'peaceful', 'composed'],
            'disgust': ['disgust', 'disgusted', 'dis', 'revulsion', 'contempt'],
            'fearful': ['fear', 'fearful', 'fea', 'afraid', 'scared', 'anxiety'],
            'happy': ['happy', 'happiness', 'hap', 'joy', 'joyful', 'pleased', 'excitement'],
            'neutral': ['neutral', 'neu', 'normal', 'baseline'],
            'sad': ['sad', 'sadness', 'sorrow', 'melancholy', 'depression'],
            'surprised': ['surprise', 'surprised', 'sur', 'amazed', 'ps', 'pleasant_surprised', 'shock']
        }

    def extract_emotion_clean(self, filepath, dataset_name):
        """Extract emotion using proven methods only"""
        self.extraction_stats['total'] += 1

        filename = os.path.basename(filepath)
        filename_lower = filename.lower()
        full_path_lower = filepath.lower()

        emotion = None

        # Dataset-specific extraction
        if 'ravdess' in dataset_name.lower():
            emotion = self._extract_ravdess(filename)
        elif 'crema' in dataset_name.lower():
            emotion = self._extract_crema(filename, full_path_lower)
        elif 'tess' in dataset_name.lower():
            emotion = self._extract_tess(filename, full_path_lower)
        elif 'emodb' in dataset_name.lower() or 'berlin' in dataset_name.lower():
            emotion = self._extract_emodb(filename)
        elif 'savee' in dataset_name.lower():
            emotion = self._extract_savee(filename)
        else:
            emotion = self._extract_universal(filename, full_path_lower)

        # Store debug info
        if len(self.debug_extractions) < 30:
            self.debug_extractions.append(f"{dataset_name}: {filename} → {emotion}")

        if emotion and emotion != 'unknown':
            self.extraction_stats['success'] += 1
            return emotion
        else:
            self.extraction_stats['failed'] += 1
            return 'unknown'

    def _extract_ravdess(self, filename):
        """RAVDESS extraction"""
        base_name = os.path.splitext(filename)[0]
        parts = base_name.split('-')

        if len(parts) >= 3:
            emotion_code = parts[2]
            ravdess_emotions = {
                '01': 'neutral', '02': 'calm', '03': 'happy',
                '04': 'sad', '05': 'angry', '06': 'fearful',
                '07': 'disgust', '08': 'surprised'
            }
            return ravdess_emotions.get(emotion_code, 'unknown')

        return 'unknown'

    def _extract_crema(self, filename, full_path):
        """CREMA-D extraction"""
        filename_upper = filename.upper()

        # Handle mixed datasets
        if any(x in full_path for x in ['tess', 'yaf_', 'oaf_']):
            return self._extract_tess(filename, full_path)
        if 'savee' in full_path:
            return self._extract_savee(filename)
        if re.match(r'\d{2}-\d{2}-\d{2}-\d{2}-\d{2}-\d{2}-\d{2}\.wav', filename):
            return self._extract_ravdess(filename)

        # CREMA-D patterns
        crema_emotions = {
            'ANG': 'angry', 'HAP': 'happy', 'SAD': 'sad',
            'NEU': 'neutral', 'FEA': 'fearful', 'DIS': 'disgust',
            'SUR': 'surprised', 'CAL': 'calm'
        }

        for code, emotion in crema_emotions.items():
            if f'_{code}_' in filename_upper or code in filename_upper:
                return emotion

        return 'unknown'

    def _extract_tess(self, filename, full_path):
        """TESS extraction"""
        filename_lower = filename.lower()

        tess_patterns = {
            '_angry': 'angry', '_anger': 'angry',
            '_disgust': 'disgust', '_disgusted': 'disgust',
            '_fear': 'fearful', '_fearful': 'fearful',
            '_happy': 'happy', '_happiness': 'happy',
            '_neutral': 'neutral',
            '_sad': 'sad', '_sadness': 'sad',
            '_ps': 'surprised',
            '_surprise': 'surprised', '_surprised': 'surprised'
        }

        for pattern, emotion in tess_patterns.items():
            if pattern in filename_lower:
                return emotion

        if '_' in filename_lower:
            parts = filename_lower.split('_')
            if len(parts) >= 3:
                emotion_part = parts[2].replace('.wav', '').replace('.mp3', '')
                for pattern, emotion in tess_patterns.items():
                    if pattern.strip('_') == emotion_part:
                        return emotion

        return 'unknown'

    def _extract_emodb(self, filename):
        """EMO-DB extraction"""
        base_name = os.path.splitext(filename)[0].upper()

        emodb_emotions = {
            'W': 'angry', 'L': 'neutral', 'E': 'disgust',
            'A': 'fearful', 'F': 'happy', 'T': 'sad', 'N': 'neutral'
        }

        positions_to_try = [-2, -3, 5, 6, 4, 3]

        for pos in positions_to_try:
            try:
                if pos < 0 and len(base_name) >= abs(pos):
                    emotion_char = base_name[pos]
                    if emotion_char in emodb_emotions:
                        return emodb_emotions[emotion_char]
                elif pos >= 0 and len(base_name) > pos:
                    emotion_char = base_name[pos]
                    if emotion_char in emodb_emotions:
                        return emodb_emotions[emotion_char]
            except:
                continue

        return 'unknown'

    def _extract_savee(self, filename):
        """SAVEE extraction"""
        base_name = os.path.splitext(filename)[0].lower()

        savee_patterns = {
            'sa': 'sad', 'su': 'surprised',
            'a': 'angry', 'h': 'happy', 'n': 'neutral',
            'f': 'fearful', 'd': 'disgust'
        }

        for pattern in ['sa', 'su']:
            if base_name.startswith(pattern) and len(base_name) > len(pattern):
                if base_name[len(pattern):].replace('_', '').isdigit():
                    return savee_patterns[pattern]

        if len(base_name) >= 1:
            first_char = base_name[0]
            if first_char in ['a', 'h', 'n', 'f', 'd'] and len(base_name) > 1:
                if base_name[1:].replace('_', '').isdigit() or base_name[1] == '_':
                    return savee_patterns[first_char]

        for speaker in ['dc', 'je', 'jk', 'kl']:
            if speaker in base_name:
                remaining = base_name.replace(speaker, '').strip('_')
                if remaining:
                    for pattern in ['sa', 'su']:
                        if remaining.startswith(pattern):
                            return savee_patterns[pattern]
                    if len(remaining) >= 1 and remaining[0] in ['a', 'h', 'n', 'f', 'd']:
                        return savee_patterns[remaining[0]]

        return 'unknown'

    def _extract_universal(self, filename, full_path):
        """Universal extraction"""
        methods = [
            self._extract_ravdess,
            self._extract_emodb,
            self._extract_savee,
        ]

        for method in methods:
            try:
                result = method(filename)
                if result != 'unknown':
                    return result
            except:
                continue

        return self._extract_tess(filename, full_path)

    def print_debug_extractions(self):
        """Print debug extractions"""
        print("\n🔍 DEBUG: Clean emotion extractions:")
        for extraction in self.debug_extractions:
            print(f"   {extraction}")

        success_rate = self.extraction_stats['success'] / max(self.extraction_stats['total'], 1) * 100
        print(f"\n📊 Clean extraction success rate: {success_rate:.1f}%")

# =============================================================================
# 5. CLEAN DATA PREPARATION
# =============================================================================

class CleanDataPreparator:
    """Clean data preparation with real audio features only"""

    def __init__(self):
        self.feature_extractor = CleanAudioFeatureExtractor()
        self.emotion_extractor = CleanEmotionExtractor()

    def prepare_clean_dataset(self, datasets, max_samples_per_emotion=1500):
        """Prepare dataset with clean audio features only"""
        print(f"\n🔧 CLEAN DATASET PREPARATION (AUDIO FEATURES ONLY)")
        print(f"Target: {max_samples_per_emotion} samples per emotion")
        print("🚫 NO synthetic features - reliable deployment")
        print("=" * 60)

        all_features = []
        all_emotions = []
        all_sources = []

        emotion_counts = {}
        processing_stats = {}

        datasets.sort(key=lambda x: x.get('priority', 999))

        for dataset_info in datasets:
            print(f"\n📊 Processing {dataset_info['name']}...")
            print(f"   🎯 Expected Performance: {dataset_info.get('accuracy', 'N/A')}")

            processed_count = 0
            failed_count = 0
            unknown_count = 0
            dataset_emotions = {}

            files_to_process = dataset_info['files']
            np.random.shuffle(files_to_process)

            max_files_per_dataset = min(len(files_to_process), 6000)
            files_to_process = files_to_process[:max_files_per_dataset]

            for audio_file in tqdm(files_to_process, desc=f"Processing {dataset_info['name']}"):
                try:
                    emotion_needs_samples = any(emotion_counts.get(emotion, 0) < max_samples_per_emotion
                                              for emotion in ['angry', 'calm', 'disgust', 'fearful', 'happy', 'neutral', 'sad', 'surprised'])

                    if not emotion_needs_samples:
                        break

                    emotion = self.emotion_extractor.extract_emotion_clean(
                        audio_file, dataset_info['name']
                    )

                    if emotion == 'unknown':
                        unknown_count += 1
                        continue

                    current_count = emotion_counts.get(emotion, 0)
                    if current_count >= max_samples_per_emotion:
                        continue

                    # Extract CLEAN audio features only
                    features = self.feature_extractor.extract_clean_audio_features(audio_file)

                    if features and len(features) > 150:  # Ensure sufficient clean audio features
                        all_features.append(list(features.values()))
                        all_emotions.append(emotion)
                        all_sources.append(dataset_info['name'])

                        emotion_counts[emotion] = emotion_counts.get(emotion, 0) + 1
                        dataset_emotions[emotion] = dataset_emotions.get(emotion, 0) + 1
                        processed_count += 1
                    else:
                        failed_count += 1

                except Exception as e:
                    failed_count += 1

            processing_stats[dataset_info['name']] = {
                'processed': processed_count,
                'failed': failed_count,
                'unknown': unknown_count,
                'emotions': dataset_emotions,
                'total_files': len(files_to_process),
                'success_rate': processed_count / len(files_to_process) if files_to_process else 0,
                'accuracy': dataset_info.get('accuracy', 'N/A')
            }

            print(f"   ✅ Processed: {processed_count} samples")
            print(f"   ❌ Failed: {failed_count} samples")
            print(f"   ❓ Unknown emotions: {unknown_count} samples")
            print(f"   📊 Success rate: {processing_stats[dataset_info['name']]['success_rate']:.1%}")
            print(f"   📊 Emotions found: {dataset_emotions}")

        self.emotion_extractor.print_debug_extractions()

        print(f"\n✅ CLEAN DATASET PREPARATION COMPLETE!")
        print(f"📊 Total samples: {len(all_features)}")
        print(f"📊 Final emotion distribution: {emotion_counts}")

        if len(all_features) == 0:
            print("❌ No features extracted!")
            return None, None, None, None

        X = np.array(all_features, dtype=np.float32)
        y = np.array(all_emotions)
        sources = np.array(all_sources)

        print(f"📊 CLEAN feature matrix shape: {X.shape}")

        return X, y, sources, emotion_counts

# =============================================================================
# 6. CLEAN MODEL TRAINING
# =============================================================================

class CleanEmotionClassifier:
    """Clean emotion classifier with audio features only"""

    def __init__(self):
        self.best_model = None
        self.scaler = RobustScaler()
        self.label_encoder = LabelEncoder()
        self.feature_selector = None

    def train_clean_models(self, X, y, test_size=0.2):
        """Train models with clean audio features only"""
        print("\n🧠 CLEAN MODEL TRAINING (AUDIO FEATURES ONLY)")
        print("🚫 NO synthetic features - reliable deployment")
        print("=" * 70)

        y_encoded = self.label_encoder.fit_transform(y)
        class_names = self.label_encoder.classes_

        print(f"📊 Classes: {list(class_names)}")
        print(f"📊 Class distribution: {dict(zip(*np.unique(y, return_counts=True)))}")

        X_train, X_test, y_train, y_test = train_test_split(
            X, y_encoded, test_size=test_size, random_state=42, stratify=y_encoded
        )

        print(f"📊 Training set: {X_train.shape[0]} samples")
        print(f"📊 Test set: {X_test.shape[0]} samples")
        print(f"📊 CLEAN audio features: {X_train.shape[1]}")

        # Feature selection
        print("\n🔍 CLEAN FEATURE SELECTION...")
        n_features = min(150, X_train.shape[1])  # Reasonable number for clean features
        self.feature_selector = SelectKBest(score_func=f_classif, k=n_features)
        X_train_selected = self.feature_selector.fit_transform(X_train, y_train)
        X_test_selected = self.feature_selector.transform(X_test)
        print(f"📊 Selected clean features: {X_train_selected.shape[1]}")

        # Scaling
        X_train_scaled = self.scaler.fit_transform(X_train_selected)
        X_test_scaled = self.scaler.transform(X_test_selected)

        # Class balancing
        print("\n⚖️ CLEAN CLASS BALANCING...")
        try:
            smote = BorderlineSMOTE(random_state=42, k_neighbors=3)
            X_train_balanced, y_train_balanced = smote.fit_resample(X_train_scaled, y_train)
            print(f"📊 Balanced training set: {X_train_balanced.shape[0]} samples")
        except:
            try:
                smote = SMOTE(random_state=42, k_neighbors=3)
                X_train_balanced, y_train_balanced = smote.fit_resample(X_train_scaled, y_train)
                print(f"📊 Balanced training set: {X_train_balanced.shape[0]} samples")
            except:
                print("   ⚠️ SMOTE failed, using original data")
                X_train_balanced = X_train_scaled
                y_train_balanced = y_train

        # Clean model ensemble (proven algorithms only)
        models = {
            'Clean XGBoost': xgb.XGBClassifier(
                n_estimators=600,
                max_depth=10,
                learning_rate=0.02,
                subsample=0.8,
                colsample_bytree=0.8,
                reg_alpha=0.1,
                reg_lambda=0.1,
                random_state=42,
                eval_metric='mlogloss',
                tree_method='hist'
            ),
            'Clean LightGBM': lgb.LGBMClassifier(
                n_estimators=600,
                max_depth=10,
                learning_rate=0.02,
                subsample=0.8,
                colsample_bytree=0.8,
                reg_alpha=0.1,
                reg_lambda=0.1,
                random_state=42,
                verbose=-1,
                objective='multiclass',
                metric='multi_logloss'
            ),
            'Clean Random Forest': RandomForestClassifier(
                n_estimators=500,
                max_depth=30,
                min_samples_split=2,
                min_samples_leaf=1,
                max_features='sqrt',
                random_state=42,
                n_jobs=-1,
                class_weight='balanced'
            ),
            'Clean SVM': SVC(
                C=10,
                gamma='scale',
                kernel='rbf',
                probability=True,
                random_state=42,
                class_weight='balanced'
            )
        }

        # Train and evaluate models
        cv_results = {}
        print(f"\n🔄 CLEAN MODEL EVALUATION:")
        print("-" * 50)

        kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

        for name, model in models.items():
            print(f"\n🧪 Training {name}...")
            try:
                cv_scores = cross_val_score(
                    model, X_train_balanced, y_train_balanced,
                    cv=kfold, scoring='f1_macro', n_jobs=1
                )

                model.fit(X_train_balanced, y_train_balanced)
                y_pred = model.predict(X_test_scaled)
                test_accuracy = accuracy_score(y_test, y_pred)
                test_f1 = f1_score(y_test, y_pred, average='macro')

                cv_results[name] = {
                    'model': model,
                    'cv_f1_mean': cv_scores.mean(),
                    'cv_f1_std': cv_scores.std(),
                    'test_accuracy': test_accuracy,
                    'test_f1': test_f1
                }

                print(f"   📊 CV F1-Score: {cv_scores.mean():.3f} ± {cv_scores.std():.3f}")
                print(f"   📊 Test Accuracy: {test_accuracy:.3f}")
                print(f"   📊 Test F1-Score: {test_f1:.3f}")

            except Exception as e:
                print(f"   ❌ Error with {name}: {e}")

        if not cv_results:
            print("❌ No models trained successfully!")
            return None, None, None, None

        # Create clean ensemble
        print(f"\n🤝 CREATING CLEAN ENSEMBLE...")

        sorted_models = sorted(cv_results.items(), key=lambda x: x[1]['test_f1'], reverse=True)
        top_models = sorted_models[:3]

        if len(top_models) >= 2:
            ensemble_models = [(name, data['model']) for name, data in top_models]

            ensemble = VotingClassifier(
                estimators=ensemble_models,
                voting='soft'
            )

            ensemble.fit(X_train_balanced, y_train_balanced)

            y_pred_ensemble = ensemble.predict(X_test_scaled)
            ensemble_accuracy = accuracy_score(y_test, y_pred_ensemble)
            ensemble_f1 = f1_score(y_test, y_pred_ensemble, average='macro')

            print(f"   📊 Clean Ensemble Accuracy: {ensemble_accuracy:.3f}")
            print(f"   📊 Clean Ensemble F1-Score: {ensemble_f1:.3f}")

            cv_results['Clean Ensemble'] = {
                'model': ensemble,
                'test_accuracy': ensemble_accuracy,
                'test_f1': ensemble_f1
            }

        # Select best model
        best_model_name = max(cv_results.keys(), key=lambda x: cv_results[x]['test_f1'])
        self.best_model = cv_results[best_model_name]['model']

        print(f"\n🏆 BEST CLEAN MODEL: {best_model_name}")
        print(f"   📊 Test Accuracy: {cv_results[best_model_name]['test_accuracy']:.3f}")
        print(f"   📊 Test F1-Score: {cv_results[best_model_name]['test_f1']:.3f}")

        # Final evaluation
        y_pred_final = self.best_model.predict(X_test_scaled)

        print(f"\n📊 DETAILED CLEAN PERFORMANCE ANALYSIS:")
        print("-" * 60)
        print(classification_report(y_test, y_pred_final, target_names=class_names, digits=3))

        return cv_results, X_test_scaled, y_test, class_names

# =============================================================================
# 7. CLEAN MODEL SAVING
# =============================================================================

def save_clean_models(classifier, analytics, feature_extractor=None, save_dir="clean_audio_models"):
    """Save clean audio-only models for deployment"""
    import os
    import joblib
    import json
    from datetime import datetime

    print(f"\n💾 SAVING CLEAN AUDIO-ONLY MODELS")
    print(f"📂 Target directory: {save_dir}")
    print("🚫 NO synthetic features - deployment ready")
    print("=" * 70)

    os.makedirs(save_dir, exist_ok=True)

    try:
        # Save clean models
        model_path = os.path.join(save_dir, 'sota_xgboost_model.pkl')
        joblib.dump(classifier.best_model, model_path)
        model_size = os.path.getsize(model_path) / (1024*1024)
        print(f"✅ Saved Clean Model: {model_path} ({model_size:.1f} MB)")

        scaler_path = os.path.join(save_dir, 'robust_scalar.pkl')
        joblib.dump(classifier.scaler, scaler_path)
        scaler_size = os.path.getsize(scaler_path) / 1024
        print(f"✅ Saved Clean Scaler: {scaler_path} ({scaler_size:.1f} KB)")

        selector_path = os.path.join(save_dir, 'feature_selector.pkl')
        joblib.dump(classifier.feature_selector, selector_path)
        selector_size = os.path.getsize(selector_path) / 1024
        print(f"✅ Saved Clean Feature Selector: {selector_path} ({selector_size:.1f} KB)")

        encoder_path = os.path.join(save_dir, 'label_encoder.pkl')
        joblib.dump(classifier.label_encoder, encoder_path)
        encoder_size = os.path.getsize(encoder_path) / 1024
        print(f"✅ Saved Clean Label Encoder: {encoder_path} ({encoder_size:.1f} KB)")

        # Generate clean feature names
        if feature_extractor and hasattr(feature_extractor, 'feature_names') and feature_extractor.feature_names:
            feature_names = feature_extractor.feature_names
        else:
            feature_names = generate_clean_feature_names()

        feature_names_path = os.path.join(save_dir, 'feature_names.pkl')
        joblib.dump(feature_names, feature_names_path)
        names_size = os.path.getsize(feature_names_path) / 1024
        print(f"✅ Saved Clean Feature Names: {feature_names_path} ({len(feature_names)} features, {names_size:.1f} KB)")

        # Save clean metadata
        metadata = {
            'model_type': f"Clean Audio-Only {analytics.get('best_model', 'Best Model')}",
            'author': 'Peter Chika Ozo-ogueji (Data Scientist)',
            'accuracy': float(analytics['test_accuracy']),
            'f1_score': float(analytics['test_f1_score']),
            'feature_count': int(analytics['feature_count']),
            'emotion_classes': list(classifier.label_encoder.classes_),
            'dataset_sources': analytics.get('dataset_sources', []),
            'sota_techniques': [
                'Clean Audio Features Only',
                'MFCC + Spectral + Prosodic',
                'No Synthetic Features',
                'Deployment Reliable'
            ],
            'training_date': datetime.now().isoformat(),
            'extraction_success_rate': float(analytics.get('extraction_success_rate', 0)),
            'total_samples': int(analytics['total_samples']),
            'best_model_name': analytics.get('best_model', 'Clean Audio Model'),
            'cv_score': float(analytics.get('cv_score', 0)),
            'feature_selection_k': 150,
            'scaling_method': 'RobustScaler',
            'cross_validation': 'StratifiedKFold',
            'deployment_ready': True,
            'streamlit_compatible': True,
            'synthetic_features_removed': True
        }

        metadata_path = os.path.join(save_dir, 'model_metadata.json')
        with open(metadata_path, 'w') as f:
            json.dump(metadata, f, indent=2)
        metadata_size = os.path.getsize(metadata_path) / 1024
        print(f"✅ Saved Clean Metadata: {metadata_path} ({metadata_size:.1f} KB)")

        total_size = sum([
            os.path.getsize(model_path),
            os.path.getsize(scaler_path),
            os.path.getsize(selector_path),
            os.path.getsize(encoder_path),
            os.path.getsize(feature_names_path),
            os.path.getsize(metadata_path)
        ]) / (1024*1024)

        print(f"\n🎉 ALL CLEAN MODELS SAVED SUCCESSFULLY!")
        print(f"📂 Location: {os.path.abspath(save_dir)}")
        print(f"📊 Total size: {total_size:.1f} MB")
        print(f"🎯 Model accuracy: {analytics['test_accuracy']:.3f}")
        print(f"📈 F1-score: {analytics['test_f1_score']:.3f}")
        print(f"🔬 Clean features: {analytics['feature_count']}")
        print(f"📚 Samples: {analytics['total_samples']:,}")
        print(f"🚫 Synthetic features: REMOVED")

        return True

    except Exception as e:
        print(f"❌ Error saving clean models: {e}")
        import traceback
        traceback.print_exc()
        return False

def generate_clean_feature_names():
    """Generate clean feature names (audio only)"""
    feature_names = []

    # MFCC features (13 × 8 = 104 features)
    for i in range(13):
        for stat in ['mean', 'std', 'max', 'min', 'skew', 'kurtosis']:
            feature_names.append(f'mfcc_{i}_{stat}')
        feature_names.append(f'mfcc_delta_{i}_mean')
        feature_names.append(f'mfcc_delta2_{i}_mean')

    # Spectral features (4 × 4 = 16 features)
    for feature in ['spectral_centroid', 'spectral_rolloff', 'spectral_bandwidth', 'zero_crossing_rate']:
        for stat in ['mean', 'std', 'max', 'skew']:
            feature_names.append(f'{feature}_{stat}')

    # Chroma features (12 × 2 = 24 features)
    for i in range(12):
        for stat in ['mean', 'std']:
            feature_names.append(f'chroma_{i}_{stat}')

    # Prosodic features (11 features)
    prosodic_features = ['f0_mean', 'f0_std', 'f0_range', 'f0_jitter', 'f0_shimmer',
                        'f0_slope', 'f0_curvature', 'energy_mean', 'energy_std', 'energy_skew', 'energy_kurtosis']
    feature_names.extend(prosodic_features)

    # Advanced spectral features (16 features)
    for i in range(7):
        feature_names.append(f'spectral_contrast_{i}_mean')
        feature_names.append(f'spectral_contrast_{i}_std')
    feature_names.extend(['spectral_flatness_mean', 'spectral_flatness_std'])

    # Harmonic features (15 features)
    for i in range(6):
        feature_names.append(f'tonnetz_{i}_mean')
        feature_names.append(f'tonnetz_{i}_std')
    feature_names.extend(['harmonic_energy', 'percussive_energy', 'harmonic_percussive_ratio'])

    # Temporal features (5 features)
    temporal_features = ['tempo', 'beat_count', 'beat_variance', 'onset_count', 'onset_rate']
    feature_names.extend(temporal_features)

    return feature_names

# =============================================================================
# 8. MAIN EXECUTION
# =============================================================================

def build_clean_production_system():
    """Build clean production system with audio features only"""
    print("🏗️ BUILDING CLEAN PRODUCTION SYSTEM")
    print("🚫 NO synthetic features - reliable deployment")
    print("👨‍💻 Author: Peter Chika Ozo-ogueji (Data Scientist)")
    print("=" * 80)

    setup_kaggle()

    datasets = download_sota_datasets()
    if not datasets:
        print("❌ No datasets downloaded")
        return None, None

    preparator = CleanDataPreparator()
    result = preparator.prepare_clean_dataset(datasets, max_samples_per_emotion=1500)

    if result[0] is None:
        print("❌ Data preparation failed")
        return None, None

    X, y, sources, emotion_counts = result

    if len(X) < 1500:
        print("❌ Insufficient data for training")
        return None, None

    classifier = CleanEmotionClassifier()
    cv_results, X_test, y_test, class_names = classifier.train_clean_models(X, y)

    if cv_results is None:
        print("❌ Model training failed")
        return None, None

    best_model_name = max(cv_results.keys(), key=lambda x: cv_results[x]['test_f1'])

    analytics = {
        'total_samples': len(X),
        'emotion_distribution': emotion_counts,
        'best_model': best_model_name,
        'test_accuracy': cv_results[best_model_name]['test_accuracy'],
        'test_f1_score': cv_results[best_model_name]['test_f1'],
        'feature_count': X.shape[1],
        'dataset_sources': list(set(sources)),
        'extraction_success_rate': preparator.emotion_extractor.extraction_stats['success'] / max(preparator.emotion_extractor.extraction_stats['total'], 1) * 100,
    }

    accuracy_pct = analytics['test_accuracy'] * 100
    f1_pct = analytics['test_f1_score'] * 100

    print(f"\n🎉 CLEAN PRODUCTION SYSTEM COMPLETE!")
    print(f"📊 Total samples: {analytics['total_samples']:,}")
    print(f"📊 Best clean model: {analytics['best_model']}")
    print(f"📊 Test accuracy: {accuracy_pct:.1f}%")
    print(f"📊 Test F1-score: {f1_pct:.1f}%")
    print(f"📊 Clean audio features: {analytics['feature_count']}")
    print(f"📊 Extraction success: {analytics['extraction_success_rate']:.1f}%")
    print(f"🚫 Synthetic features: REMOVED")

    if accuracy_pct >= 80:
        print("🎯 ✅ EXCELLENT PERFORMANCE: 80%+ ACCURACY!")
    elif accuracy_pct >= 75:
        print("🎯 🔶 GOOD PERFORMANCE: 75%+ ACCURACY!")
    else:
        print(f"🎯 📈 BASELINE PERFORMANCE: {accuracy_pct:.1f}%")

    return classifier, analytics

def main():
    """Main execution function"""
    print("🚀 STARTING CLEAN AUDIO-ONLY SYSTEM...")
    print("🚫 NO synthetic features - reliable deployment")
    print("👨‍💻 Author: Peter Chika Ozo-ogueji (Data Scientist)")

    try:
        clean_classifier, clean_analytics = build_clean_production_system()

        print("\n" + "="*80)
        print("🎉 CLEAN AUDIO-ONLY SPEECH ANALYTICS COMPLETE!")
        print("🚫 NO SYNTHETIC FEATURES - DEPLOYMENT READY!")
        print("🎯 RELIABLE PERFORMANCE WITH REAL AUDIO FEATURES!")
        print("👨‍💻 Author: Peter Chika Ozo-ogueji (Data Scientist)")
        print("="*80)

        if clean_classifier and clean_analytics:
            print("\n" + "="*80)
            print("💾 SAVING CLEAN MODELS FOR DEPLOYMENT")
            print("="*80)

            try:
                preparator = CleanDataPreparator()
                feature_extractor = preparator.feature_extractor
            except:
                feature_extractor = None

            save_success = save_clean_models(
                classifier=clean_classifier,
                analytics=clean_analytics,
                feature_extractor=feature_extractor,
                save_dir="clean_audio_models"
            )

            if save_success:
                print("\n🎉 CLEAN MODELS SAVED SUCCESSFULLY!")
                print("🚫 NO synthetic features - deployment ready!")
                print("🚀 Your clean audio model should work reliably!")
            else:
                print("\n❌ Failed to save clean models")

        return clean_classifier, clean_analytics

    except Exception as e:
        print(f"❌ System error: {e}")
        import traceback
        traceback.print_exc()
        return None, None

if __name__ == "__main__":
    main()

🚀 CLEAN AUDIO-ONLY SOTA SPEECH ANALYTICS
🎯 NO SYNTHETIC FEATURES - RELIABLE DEPLOYMENT
👨‍💻 Author: Peter Chika Ozo-ogueji (Data Scientist)
🔧 FIXED: Removes problematic synthetic features
✅ All packages imported successfully!
🚀 STARTING CLEAN AUDIO-ONLY SYSTEM...
🚫 NO synthetic features - reliable deployment
👨‍💻 Author: Peter Chika Ozo-ogueji (Data Scientist)
🏗️ BUILDING CLEAN PRODUCTION SYSTEM
🚫 NO synthetic features - reliable deployment
👨‍💻 Author: Peter Chika Ozo-ogueji (Data Scientist)
🔑 Setting up Kaggle API...
✅ Kaggle API configured!

📥 DOWNLOADING PROVEN AUDIO EMOTION DATASETS
🎯 Focus: Real audio signal processing only

📊 Downloading RAVDESS Audio-Only...
   🎯 Expected Performance: 85%+ with clean features
   ✅ Total audio files: 2880

📊 Downloading CREMA-D Audio...
   🎯 Expected Performance: 80%+ validated
   ✅ Total audio files: 12162

📊 Downloading TESS Clean Audio...
   🎯 Expected Performance: High performance
   ✅ Total audio files: 5600

📊 Downloading EMO-DB German...
   

Processing RAVDESS Audio-Only:   0%|          | 1/2880 [00:27<21:46:26, 27.23s/it]

✅ Extracting 191 CLEAN audio features per file


Processing RAVDESS Audio-Only: 100%|██████████| 2880/2880 [19:08<00:00,  2.51it/s]


   ✅ Processed: 2880 samples
   ❌ Failed: 0 samples
   ❓ Unknown emotions: 0 samples
   📊 Success rate: 100.0%
   📊 Emotions found: {'neutral': 192, 'happy': 384, 'sad': 384, 'calm': 384, 'fearful': 384, 'disgust': 384, 'surprised': 384, 'angry': 384}

📊 Processing CREMA-D Audio...
   🎯 Expected Performance: 80%+ validated


...

   ✅ Processed: 2102 samples
   ❌ Failed: 0 samples
   ❓ Unknown emotions: 0 samples
   📊 Success rate: 37.5%
   📊 Emotions found: {'surprised': 798, 'fearful': 141, 'neutral': 454, 'happy': 177, 'disgust': 180, 'angry': 168, 'sad': 184}

📊 Processing EMO-DB German...
   🎯 Expected Performance: 90%+ on clean features


Processing EMO-DB German: 100%|██████████| 535/535 [00:00<00:00, 138892.83it/s]


   ✅ Processed: 0 samples
   ❌ Failed: 0 samples
   ❓ Unknown emotions: 0 samples
   📊 Success rate: 0.0%
   📊 Emotions found: {}

📊 Processing SAVEE British...
   🎯 Expected Performance: Proven benchmark


Processing SAVEE British: 100%|██████████| 960/960 [00:00<00:00, 185358.00it/s]


   ✅ Processed: 0 samples
   ❌ Failed: 0 samples
   ❓ Unknown emotions: 0 samples
   📊 Success rate: 0.0%
   📊 Emotions found: {}

🔍 DEBUG: Clean emotion extractions:
   RAVDESS Audio-Only: 03-01-01-01-02-02-14.wav → neutral
   RAVDESS Audio-Only: 03-01-03-01-02-02-21.wav → happy
   RAVDESS Audio-Only: 03-01-04-02-02-01-05.wav → sad
   RAVDESS Audio-Only: 03-01-02-02-02-01-12.wav → calm
   RAVDESS Audio-Only: 03-01-04-02-02-01-20.wav → sad
   RAVDESS Audio-Only: 03-01-06-01-01-01-10.wav → fearful
   RAVDESS Audio-Only: 03-01-07-02-01-02-23.wav → disgust
   RAVDESS Audio-Only: 03-01-04-02-01-02-19.wav → sad
   RAVDESS Audio-Only: 03-01-02-02-01-02-15.wav → calm
   RAVDESS Audio-Only: 03-01-04-01-02-02-23.wav → sad
   RAVDESS Audio-Only: 03-01-08-02-02-01-16.wav → surprised
   RAVDESS Audio-Only: 03-01-05-01-01-02-16.wav → angry
   RAVDESS Audio-Only: 03-01-06-02-01-02-16.wav → fearful
   RAVDESS Audio-Only: 03-01-08-02-01-01-23.wav → surprised
   RAVDESS Audio-Only: 03-01-02-02-01-01-22